In [226]:
import gzip
import csv
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score

In [48]:
def gen_recipe_info():
    rInfo = {}
    fileName = 'RAW_recipes_textProc_VecTag(th=5)_VecIng(th=500).csv'
    with open(fileName, mode='r', encoding='utf-8') as fr:
        reader = csv.DictReader(fr)
        for row in reader:
            recipeId = row['id']
            recipeInfo = [row['name'], row['minutes'], row['tags'], row['nutrition'], row['n_steps'], \
                          row['description'], row['ingredients'], row['n_ingredients']]
            rInfo[recipeId] = recipeInfo
    return rInfo

In [166]:
def remove_chars(str):
    removeChars = " '\"\\[]'"
    return ''.join(ch for ch in str if ch not in removeChars)

In [50]:
# [0]: name, [1]: minutes, [2]: tags, [3]: nutrition, [4]: n_steps, [5]: description, [6]: ingredients, [7]: n_ingredients
rInfo = gen_recipe_info()

In [56]:
allRatings = []
with open('RAW_interactions_textProc.csv', mode='r', encoding='utf-8') as fr:
    reader = csv.DictReader(fr)
    for row in reader:
        allRatings.append(row)

In [132]:
with open('RAW_interactions_textProc.csv', mode='r', encoding='utf-8') as fr, \
    open('RAW_interactions_textProc_training.csv', mode='w', encoding='utf-8', newline='') as fw_tr, \
    open('RAW_interactions_textProc_valid.csv', mode='w', encoding='utf-8', newline='') as fw_va, \
    open('RAW_interactions_textProc_test.csv', mode='w', encoding='utf-8', newline='') as fw_te:
        reader = csv.DictReader(fr)
        writer1 = csv.DictWriter(fw_tr, fieldnames=reader.fieldnames)
        writer1.writeheader()
        writer2 = csv.DictWriter(fw_va, fieldnames=reader.fieldnames)
        writer2.writeheader()
        writer3 = csv.DictWriter(fw_te, fieldnames=reader.fieldnames)
        writer3.writeheader()
        cnt = 0
        for row in reader:
            if cnt < 960000:
                writer1.writerow(row)
                cnt += 1
            elif cnt < 1045000:
                writer2.writerow(row)
                cnt += 1
            else:
                writer3.writerow(row)

In [60]:
len(allRatings)

1132367

In [110]:
ratingsTrain = allRatings[:960000]
ratingsValid = allRatings[960000:1045000]
ratingsTest = allRatings[1045000:]

In [216]:
# 1. Minutes
X1 = [[1, int(rInfo[d['recipe_id']][1])] for d in ratingsTrain]
Y1 = [int(d['rating']) for d in ratingsTrain]
q1_model = linear_model.LinearRegression()
q1_model.fit(X1, Y1)
print(q1_model.coef_)
print(q1_model.intercept_)

[0.00000000e+00 1.51987753e-10]
4.409906022277868


In [236]:
Y1_predict = q1_model.predict(X1)
mse1 = mean_squared_error(Y1, Y1_predict)
mae1 = mean_absolute_error(Y1, Y1_predict)
r2_1 = r2_score(Y1, Y1_predict)

In [238]:
print(q1_model.coef_)
print(q1_model.intercept_)

[0.00000000e+00 1.51987753e-10]
4.409906022277868


In [232]:
print(mse1, mae1, r2_1)

1.6036571339551189 0.8505602587076813 1.3147360585774592e-06


In [172]:
remove_chars(rInfo['1'][3]).split(',')

['173.4', '18.0', '0.0', '17.0', '22.0', '35.0', '1.0']

In [190]:
# 2-1. Nutrition rInfo[id][3](calories, ?, ?, ?, ?, ?, ?)

In [250]:
X2 = [
    [1] + list(map(float, remove_chars(rInfo[d['recipe_id']][3]).split(',')))
    for d in ratingsTrain
]
Y2 = [int(d['rating']) for d in ratingsTrain]

In [252]:
q2_model = linear_model.LinearRegression()
q2_model.fit(X2, Y2)

LinearRegression()

In [254]:
Y2_predict = q2_model.predict(X2)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)

In [256]:
print(q2_model.coef_)
print(q2_model.intercept_)

[ 0.00000000e+00 -1.28746570e-05  1.82581182e-04  4.91643016e-05
 -1.26615039e-05  8.87159843e-06 -8.80744964e-05 -5.37975720e-04]
4.417661130959591


In [258]:
print(mse2, mae2, r2_2)

1.603316745954154 0.8504751741624994 0.00021357179914116564


In [260]:
# nutrition[0] only
X2_0 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[0])]
    for d in ratingsTrain
]

In [262]:
q2_0_model = linear_model.LinearRegression()
q2_0_model.fit(X2_0, Y2)
print(q2_0_model.coef_)
print(q2_0_model.intercept_)

[ 0.00000000e+00 -1.43140206e-05]
4.416523717116527


In [270]:
Y2_predict = q2_0_model.predict(X2_0)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6034179642361324 0.8505619989612618 0.00015045472332730903


In [274]:
# nutrition[1] only
X2_1 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[1])]
    for d in ratingsTrain
]
q2_1_model = linear_model.LinearRegression()
q2_1_model.fit(X2_1, Y2)
print(q2_1_model.coef_)
print(q2_1_model.intercept_)

[ 0.000000e+00 -8.489012e-05]
4.4128638038823205


In [276]:
Y2_predict = q2_1_model.predict(X2_1)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6036189238677374 0.8505964884640874 2.5141548122276092e-05


In [278]:
# nutrition[2] only
X2_2 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[2])]
    for d in ratingsTrain
]
q2_2_model = linear_model.LinearRegression()
q2_2_model.fit(X2_2, Y2)
print(q2_2_model.coef_)
print(q2_2_model.intercept_)

[ 0.00000000e+00 -2.02887575e-05]
4.411523869533387


In [280]:
Y2_predict = q2_2_model.predict(X2_2)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6034670318966364 0.8505681819269696 0.00011985741237219294


In [286]:
# nutrition[3] only
X2_3 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[3])]
    for d in ratingsTrain
]
q2_3_model = linear_model.LinearRegression()
q2_3_model.fit(X2_3, Y2)
print(q2_3_model.coef_)
print(q2_3_model.intercept_)

[ 0.00000000e+00 -3.89592969e-05]
4.411171060727147


In [287]:
Y2_predict = q2_3_model.predict(X2_3)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.603635186160851 0.850583703099298 1.5000807069398547e-05


In [290]:
# nutrition[4] only
X2_4 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[4])]
    for d in ratingsTrain
]
q2_4_model = linear_model.LinearRegression()
q2_4_model.fit(X2_4, Y2)
print(q2_4_model.coef_)
print(q2_4_model.intercept_)

[ 0.0000000e+00 -6.9214276e-05]
4.412398743988542


In [292]:
Y2_predict = q2_4_model.predict(X2_4)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6036298545563723 0.850533876563105 1.8325456307866084e-05


In [294]:
# nutrition[5] only
X2_5 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[5])]
    for d in ratingsTrain
]
q2_5_model = linear_model.LinearRegression()
q2_5_model.fit(X2_5, Y2)
print(q2_5_model.coef_)
print(q2_5_model.intercept_)

[ 0.00000000e+00 -7.68902422e-05]
4.413321258607433


In [296]:
Y2_predict = q2_5_model.predict(X2_5)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6036033345367045 0.8506048407568262 3.486264760610425e-05


In [298]:
# nutrition[6] only
X2_6 = [
    [1, float(remove_chars(rInfo[d['recipe_id']][3]).split(',')[6])]
    for d in ratingsTrain
]
q2_6_model = linear_model.LinearRegression()
q2_6_model.fit(X2_6, Y2)
print(q2_6_model.coef_)
print(q2_6_model.intercept_)

[ 0.         -0.00022891]
4.413341925813723


In [300]:
Y2_predict = q2_6_model.predict(X2_6)
mse2 = mean_squared_error(Y2, Y2_predict)
mae2 = mean_absolute_error(Y2, Y2_predict)
r2_2 = r2_score(Y2, Y2_predict)
print(mse2, mae2, r2_2)

1.6033997366304895 0.8505378210118469 0.00016182098191996896
